# Model Training and Evaluation
This notebook focuses on continuing the training of the [sequence models](sequence_modeling.ipynb) (LSTM and RNN), evaluating their performance, and documenting the results.


In [ ]:
# Setup and Imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt

## Load Data and Models

In [ ]:
# Load preprocessed data
data = pd.read_csv('data/preprocessed_data.csv')

# Split features and labels
X = data.drop('activity_label', axis=1).values
y = data['activity_label'].values

# Reshape data for LSTM and RNN input
time_steps = 100  # Example time step length
num_features = X.shape[1] // time_steps
X = X.reshape((-1, time_steps, num_features))

# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the saved LSTM model
lstm_model = load_model('models/lstm_model.h5')

# Load the saved RNN model
rnn_model = load_model('models/rnn_model.h5')

## Extended Training

In [ ]:
# Continue training the LSTM model
lstm_history = lstm_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=2)

# Save the trained LSTM model
lstm_model.save('models/final_lstm_model.h5')

In [ ]:
# Continue training the RNN model
rnn_history = rnn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=2)

# Save the trained RNN model
rnn_model.save('models/final_rnn_model.h5')

## Model Evaluation

In [ ]:
# Predict on test data using LSTM model
lstm_y_pred = lstm_model.predict(X_test)
lstm_y_pred_classes = np.argmax(lstm_y_pred, axis=1)

# Calculate performance metrics for LSTM model
lstm_accuracy = accuracy_score(y_test, lstm_y_pred_classes)
lstm_precision = precision_score(y_test, lstm_y_pred_classes, average='weighted')
lstm_recall = recall_score(y_test, lstm_y_pred_classes, average='weighted')

print(f'LSTM Model Accuracy: {lstm_accuracy}')
print(f'LSTM Model Precision: {lstm_precision}')
print(f'LSTM Model Recall: {lstm_recall}')

# Confusion matrix for LSTM model
lstm_conf_matrix = confusion_matrix(y_test, lstm_y_pred_classes)
print('LSTM Model Confusion Matrix:')
print(lstm_conf_matrix)

In [ ]:
# Predict on test data using RNN model
rnn_y_pred = rnn_model.predict(X_test)
rnn_y_pred_classes = np.argmax(rnn_y_pred, axis=1)

# Calculate performance metrics for RNN model
rnn_accuracy = accuracy_score(y_test, rnn_y_pred_classes)
rnn_precision = precision_score(y_test, rnn_y_pred_classes, average='weighted')
rnn_recall = recall_score(y_test, rnn_y_pred_classes, average='weighted')

print(f'RNN Model Accuracy: {rnn_accuracy}')
print(f'RNN Model Precision: {rnn_precision}')
print(f'RNN Model Recall: {rnn_recall}')

# Confusion matrix for RNN model
rnn_conf_matrix = confusion_matrix(y_test, rnn_y_pred_classes)
print('RNN Model Confusion Matrix:')
print(rnn_conf_matrix)

## Visualization of Results

In [ ]:
# Plot training & validation accuracy values for LSTM
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(lstm_history.history['accuracy'])
plt.plot(lstm_history.history['val_accuracy'])
plt.title('LSTM Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values for LSTM
plt.subplot(1, 2, 2)
plt.plot(lstm_history.history['loss'])
plt.plot(lstm_history.history['val_loss'])
plt.title('LSTM Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
# Plot training & validation accuracy values for RNN
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(rnn_history.history['accuracy'])
plt.plot(rnn_history.history['val_accuracy'])
plt.title('RNN Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values for RNN
plt.subplot(1, 2, 2)
plt.plot(rnn_history.history['loss'])
plt.plot(rnn_history.history['val_loss'])
plt.title('RNN Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

## Conclusion
This notebook continued the training of the LSTM and RNN models, evaluated their performance, and documented the results. The models were assessed using accuracy, precision, recall, and confusion matrix metrics. The training processes and results were visualized to provide a clear understanding of the models' performance.